# Caderno 3. Pesquisa chunks relacionados às questões

Esse caderno apenas pesquisa, para cada query, quais são os 500 primeiros resultados.

Para os testes nos próximos cadernos, não usaremos os 500 primeiros resultados. Provavelmente usaremos apenas de 0 a 5 resultados. No entanto, como é possível que tenhamos que fazer testes com exemplos pouco relacionados à consulta, vou salvar os 500 primeiros resultados mesmo. Caso sejam irrelevantes, basta desconsiderá-los.

In [1]:
import json
import pandas as pd
import faiss
from tqdm import tqdm
import h5py
import numpy as np
import pickle
import gzip

from bm25 import IndiceInvertido, BM25, tokenizador_pt_remove_html

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Total de registros para buscar e local de salvamento.

In [2]:
k = 500

NOME_ARQUIVO_RESULTADOS_PESQUISAS_TODOS_CHUNKS = 'outputs/3 - resultados_pesquisas/resultados_pesquisas_todos_chunks.pickle.gz'
NOME_ARQUIVO_RESULTADOS_PESQUISAS_APENAS_ART = 'outputs/3 - resultados_pesquisas/resultados_pesquisas_apenas_art.pickle.gz'

Nome dos índices para pesquisa léxica.

In [3]:
NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS = 'outputs/1 - indices_invertidos/indice_bm25_todos_chunks.pickle'
NOME_ARQUIVO_INDICE_BM25_APENAS_ART = 'outputs/1 - indices_invertidos/indice_bm25_apenas_art.pickle'

Nome dos modelos utilizados para pesquisa semântica.

In [4]:
NOME_ARQUIVO_EMBEDDINGS_CHUNKS = 'outputs/2 - embeddings/embeddings_chunks.h5'
NOME_ARQUIVO_EMBEDDINGS_QUESTOES = 'outputs/2 - embeddings/embeddings_questoes.h5'

NOME_MODELO_EMB_OPENAI = "text-embedding-3-large"

# Modelos disponíveis
NOME_MODELOS_EMB = [NOME_MODELO_EMB_OPENAI]

# 1. Carregar as bases de dados de chunks e de questões

In [19]:
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

chunks_pesquisa = load_jsonl('inputs/chunks_pesquisa.jsonl')
questoes = load_jsonl('inputs/questoes.jsonl')

# 2. Pesquisas

Dicionários onde serão salvos os resultados das pesquisas. São dois dicionários, um para todos os chunks e outro para apenas os chunks de artigos.

Ele será populado da seguinte forma:

<code>resultados_pesquisas = {
   "id_consulta": {
       "modelo": {
           urn: ["", ..., ""],
           score: ["", ..., ""]
       }
   }
}</code>

onde modelo é o modelo utilizado (por exemplo, bm25, algum modelo de embeddings etc).

In [6]:
resultados_pesquisas_todos_chunks = {}
resultados_pesquisas_apenas_art = {}

for q in questoes:
    id_questao = q['ID_QUESTAO']
    resultados_pesquisas_todos_chunks[id_questao] = {}
    resultados_pesquisas_apenas_art[id_questao] = {}

## 2.1. Pesquisa léxica - BM25

Recupera os índices invertidos gerados no caderno 1.

In [7]:
# Índice para todos os chunks
iidx_todos_chunks = IndiceInvertido(tokenizador_pt_remove_html)
iidx_todos_chunks.from_pickle(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS)

# Índice para os chunks apenas de artigos
iidx_apenas_art = IndiceInvertido(tokenizador_pt_remove_html)
iidx_apenas_art.from_pickle(NOME_ARQUIVO_INDICE_BM25_APENAS_ART)

Cria instâncias para os buscadores

In [8]:
# Agora instancia um BM25
buscador_todos_chunks = BM25(iidx_todos_chunks, k1=0.82, b=0.68, bias_idf=1)
buscador_apenas_art = BM25(iidx_apenas_art, k1=0.82, b=0.68, bias_idf=1)

Pesquisa todas as queries e salva os resultados nos dicionários.

In [9]:
for q in tqdm(questoes):
    id_questao = q['ID_QUESTAO']
    questao = q['ENUNCIADO_COM_ALTERNATIVAS']

    resultado_query_todos_chunks = buscador_todos_chunks.pesquisar(questao)
    urn, score = zip(*resultado_query_todos_chunks[:k])
    resultados_pesquisas_todos_chunks[id_questao]['bm25'] = { 'urn': urn, 'score': score }
    
    resultado_query_apenas_art = buscador_apenas_art.pesquisar(questao)
    urn, score = zip(*resultado_query_apenas_art[:k])
    resultados_pesquisas_apenas_art[id_questao]['bm25'] = { 'urn': urn, 'score': score }    

100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [00:53<00:00, 13.00it/s]


## 2.2 Pesquisas semânticas

### 2.2.1 Funções para carregar e normalizar embeddings

Funções genéricas para carregar e normalizar os embeddings do arquivo h5.

A ideia é que há apenas dois arquivos, um para os embeddings das questões e outro para os embeddings dos chunks.


O:.: Os embeddings da OpenAI já são normalizados (mas há uma perda na conversão de f32 pra f16 para salvar no H5), nem precisaria disso.

In [10]:
def carregar_embeddings_do_arquivo(arquivo, coluna_id, nome_modelo):
    with h5py.File(arquivo, "r") as f:
        ids = f[coluna_id][:].astype(str)
        emb = f[nome_modelo][:].astype(np.float32)

    return ids, emb

def normalizar_embeddings(x):
    faiss.normalize_L2(x)
    return x

### 2.2.2 Funções para filtrar chunks (todos os chunks ou apenas representando artigos completos e jurisprudências)

Grupo de funções para filtrar os embeddings dos chunks.

Os embeddings foram gerados para todos os chunks. Como serão feitos dois grupos de testes (com todos os chunks e com chunks apenas de artigos), é necessário fazer essa filtragem.

In [11]:
# A ideia desse grupo de funções é, uma vez carregado todos os embeddings, filtrar apenas aqueles
# que batem com a máscara. Será usado para indexar apenas os embeddings que se referem a artigos completos
urns_chunks_pesquisa_apenas_art = [c['URN'] for c in chunks_pesquisa if c['TIPO'] == 'ART' or c['TIPO'] == 'JUR']
def filtro_apenas_art(urn):
    return urn in urns_chunks_pesquisa_apenas_art

def filtro_seleciona_tudo(_):
    return True
    
def filtrar_embeddings(urns, emb, func_filtro):
    mask = np.array([func_filtro(i) for i in urns], dtype=bool)

    return urns[mask], emb[mask]

### 2.2.3 Criação de índices FAISS para todos os modelos

Função para criar um índice FAISS.

In [12]:
def criar_indice_faiss(nome_modelo, filtro_seleciona_urn):
    urns, emb = carregar_embeddings_do_arquivo(NOME_ARQUIVO_EMBEDDINGS_CHUNKS, 'urn', nome_modelo)
    urns, emb = filtrar_embeddings(urns, emb, filtro_seleciona_urn)
    emb = normalizar_embeddings(emb)
    dim = emb.shape[1]
        
    index_faiss = faiss.IndexFlatL2(dim)
    index_faiss.add(emb)

    return urns, index_faiss

Cria os índices FAISS para cada modelo.

Como serão testados mais de um modelo de embeddings, será criado um mapa de índice no seguinte formato:

<code>mapa_indice_faiss = {
   'nome_modelo_1': {'urn': lista_de_urns, 'indice': indice_faiss),
   'nome_modelo_...': {'urn': lista_de_urns, 'indice': indice_faiss),
   'nome_modelo_n': {'urn': lista_de_urns, 'indice': indice_faiss)
}</code>

Obs.: Devido à forma como os embeddings são criados no caderno 2, todas as urns estão na mesma sequência. Então não precisaria de salvar uma lista de urn para cada modelo (poderíamos apenas guardar. Mas vamos deixar dessa forma, pois futuramente a geração de embeddings pode mudar. Além disso, esse índice é criado apenas na memória e essas listas de urns ocupam pouco espaço.

In [13]:
mapa_indice_faiss_todos_chunks = {}
mapa_indice_faiss_apenas_art = {}

for nome_modelo in tqdm(NOME_MODELOS_EMB):
    urns, indice = criar_indice_faiss(nome_modelo, filtro_seleciona_tudo)
    mapa_indice_faiss_todos_chunks[nome_modelo] = {'urn': urns, 'indice': indice}

    urns, indice = criar_indice_faiss(nome_modelo, filtro_apenas_art)
    mapa_indice_faiss_apenas_art[nome_modelo] = {'urn': urns, 'indice': indice}

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.37it/s]


### 2.2.4 Mapa de embeddings das questões (queries) por modelo e normalização dos embeddings

Cria mapa dos embeddings das questões por modelo.

Como serão testados mais de um modelo de embeddings, será criado um mapa da seguinte forma:

<code>mapa_emb_questao = {
   'nome_modelo_1':{'id': lista_de_idss,'emb': lista_de_embeddingse),
   'nome_modelo_...'{'id': lista_de_ids, 'emb': lista_de_embeddings),),
   'nome_modelo_n{'id': lista_de_ids, 'emb': lista_de_embeddings),ce)
}

Assim como para o mapa de índice (e urns dos chunks de pesquisa), dbs.: Devido à forma como os embeddings são criados no caderno 2, todids das questõess urns estão na mesma sequência. Então não precisaria de salvar uma lisidde urn para cada uarda, assim como foi feito antes,r. Mas vamos deixar dessa aqui também.spaço.

In [14]:
mapa_emb_questao = {}

for nome_modelo in tqdm(NOME_MODELOS_EMB):
    ids_q, emb_q = carregar_embeddings_do_arquivo(NOME_ARQUIVO_EMBEDDINGS_QUESTOES, 'id', nome_modelo)
    emb_q = normalizar_embeddings(emb_q)
    mapa_emb_questao[nome_modelo] = {'id': ids_q, 'emb': emb_q}

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 25.14it/s]


Agora pesquisa todas as queries em todos os modelos e salva nos resultados.

In [15]:
total_porcentagem = len(NOME_MODELOS_EMB)*len(questoes)

def get_urn_score(lista_urns_indexadas, distancias, indices_retornados):
    # As distâncias e os índices retornados são no shape (1, k). Primeiro, transforma tudo em lista de tamanho k:
    distancias = list(distancias[0])
    indices_retornados = indices_retornados = list(indices_retornados[0])
    
    urn = []
    score = []
    
    for d, i in zip(distancias, indices_retornados):
        urn.append(lista_urns_indexadas[i])
        score.append(1 - d/2) # O score dessa forma é a similaridade de cosseno
        
    return urn, score
    
with tqdm(total=total_porcentagem) as pbar:
    for nome_modelo in NOME_MODELOS_EMB:
        ids_q, embs_q = mapa_emb_questao[nome_modelo]['id'], mapa_emb_questao[nome_modelo]['emb']

        for idx, id_questao in enumerate(ids_q):
            emb_questao = embs_q[idx:idx+1] # shape (1, dim)

            # Consulta no índice de todos os chunks
            distancias, indices_retornados = mapa_indice_faiss_todos_chunks[nome_modelo]['indice'].search(emb_questao, k)
            urn, score = get_urn_score(mapa_indice_faiss_todos_chunks[nome_modelo]['urn'], distancias, indices_retornados)
            resultados_pesquisas_todos_chunks[id_questao][nome_modelo] = { 'urn': urn, 'score': score }

            # Consulta no índice com apenas os artigos
            distancias, indices_retornados = mapa_indice_faiss_apenas_art[nome_modelo]['indice'].search(emb_questao, k)            
            urn, score = get_urn_score(mapa_indice_faiss_apenas_art[nome_modelo]['urn'], distancias, indices_retornados)
            resultados_pesquisas_apenas_art[id_questao][nome_modelo] = { 'urn': urn, 'score': score }

            pbar.update(1)

100%|███████████████████████████████████████████████████████████████████████████████| 700/700 [00:06<00:00, 110.72it/s]


# 3. Teste: resultados por modelo por questão:

Testes

In [16]:
id_questao = '675'
mostrar_top_k = 5
todos_modelos = resultados_pesquisas_apenas_art['1'].keys()

print('***** Considerando todos os chunks *****')
for modelo in todos_modelos:
    print(f'\t{modelo}')
    top_k_urn = resultados_pesquisas_todos_chunks[id_questao][modelo]['urn'][:mostrar_top_k]
    top_k_score = resultados_pesquisas_todos_chunks[id_questao][modelo]['score'][:mostrar_top_k]
    for urn, score in zip(top_k_urn, top_k_score):
        print(f'\t\tURN: {urn} ({score:.3f})')

print('\n***** Considerando apenas os chunks de artigos *****')
for modelo in todos_modelos:
    print(f'\t{modelo}')
    top_k_urn = resultados_pesquisas_apenas_art[id_questao][modelo]['urn'][:mostrar_top_k]
    top_k_score = resultados_pesquisas_apenas_art[id_questao][modelo]['score'][:mostrar_top_k]
    for urn, score in zip(top_k_urn, top_k_score):
        print(f'\t\tURN: {urn} ({score:.3f})')

***** Considerando todos os chunks *****
	bm25
		URN: tema_stf_483 (164.937)
		URN: urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!cap2_sec2 (113.247)
		URN: urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!art9 (111.581)
		URN: urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-16;18;anexo.1!cap2 (109.955)
		URN: urn:lex:br:federal:lei:2011-11-18;12527!cap2 (100.357)
	text-embedding-3-large
		URN: tema_stf_483 (0.598)
		URN: urn:lex:br:federal:lei:2018-08-14;13709!art23_cpt (0.531)
		URN: urn:lex:br:federal:lei:2018-08-14;13709!art23_cpt_inc4 (0.522)
		URN: urn:lex:br:federal:lei:2018-08-14;13709!art23_cpt_inc2 (0.518)
		URN: urn:lex:br:federal:lei:2018-08-14;13709!art23_cpt_inc1 (0.512)

***** Considerando apenas os chunks de artigos *****
	bm25
		URN: tema_stf_483 (146.101)
		URN: urn:lex:br:autoridade.nacional.protecao.dados;conselho.diretor:resolucao:2024-07-

# 4. Salva os resultados em arquivos pickle

In [17]:
with gzip.open(NOME_ARQUIVO_RESULTADOS_PESQUISAS_TODOS_CHUNKS, 'wb') as f:
    pickle.dump(resultados_pesquisas_todos_chunks, f, protocol=pickle.HIGHEST_PROTOCOL)

with gzip.open(NOME_ARQUIVO_RESULTADOS_PESQUISAS_APENAS_ART, 'wb') as f:
    pickle.dump(resultados_pesquisas_apenas_art, f, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
# Para abrir os arquivos depois:
def load_pickle_gzip(path):
    import pickle
    import gzip
    with gzip.open(path, 'rb') as f:
        return pickle.load(f)

# temp = load_pickle_gzip(NOME_ARQUIVO_RESULTADOS_PESQUISAS_TODOS_CHUNKS)